In [1]:
!pip install fastt5 

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("flax-community/t5-base-wikisplit")
model = AutoModelForSeq2SeqLM.from_pretrained("flax-community/t5-base-wikisplit")
complex_sentence = "This comedy drama is produced by Tidy , the company she co-founded in 2008 with her husband David Peet , who is managing director ."
sample_tokenized = tokenizer(complex_sentence, return_tensors="pt")
answer = model.generate(sample_tokenized['input_ids'], attention_mask = sample_tokenized['attention_mask'], max_length=256, num_beams=5)
gene_sentence = tokenizer.decode(answer[0], skip_special_tokens=True)
gene_sentence

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1818: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


'This comedy drama is produced by Tidy. She co-founded Tidy in 2008 with her husband David Peet, who is managing director.'

In [12]:
def split_sentence(complex_sentence, model)->str:
  sample_tokenized = tokenizer(complex_sentence, return_tensors="pt")
  answer = model.generate(sample_tokenized['input_ids'], attention_mask = sample_tokenized['attention_mask'], max_length=256, num_beams=5)
  gene_sentence = tokenizer.decode(answer[0], skip_special_tokens=True)
  return gene_sentence

In [4]:
split_sentence("This task is based on the WikiSplit dataset consisting of one million English sentences based on edits from Wikipedia’s revision history.", model, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1818: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


'This task is based on the WikiSplit dataset consisting of one million English sentences. It is based on edits from Wikipedia’s revision history.'

In [5]:
from fastT5 import (OnnxT5, get_onnx_runtime_sessions,
                    generate_onnx_representation, quantize)
from transformers import AutoTokenizer

model_or_model_path = 'flax-community/t5-base-wikisplit'

In [7]:
# Step 1. convert huggingfaces t5 model to onnx
onnx_model_paths = generate_onnx_representation(model_or_model_path)

Exporting to onnx... |################################| 3/3


In [8]:
# Step 2. (recommended) quantize the converted model for fast inference and to reduce model size.
quant_model_paths = quantize(onnx_model_paths)

Quantizing... |################################| 3/3


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
quant_model_paths = ('/content/gdrive/MyDrive/t5-base-wikisplit/t5-base-wikisplit-encoder-quantized.onnx',
 '/content/gdrive/MyDrive/t5-base-wikisplit/t5-base-wikisplit-decoder-quantized.onnx',
 '/content/gdrive/MyDrive/t5-base-wikisplit/t5-base-wikisplit-init-decoder-quantized.onnx')

In [8]:
# step 3. setup onnx runtime
model_sessions = get_onnx_runtime_sessions(quant_model_paths)

In [9]:
# step 4. get the onnx model
model_onnx = OnnxT5(model_or_model_path, model_sessions)

In [11]:
# step 5. test that out!
t_input = "This task is based on the WikiSplit dataset consisting of one million English sentences based on edits from Wikipedia’s revision history."
token = tokenizer(t_input, return_tensors='pt')

tokens = model_onnx.generate(input_ids=token['input_ids'],
               attention_mask=token['attention_mask'],
               num_beams=5, max_length=256)

output = tokenizer.decode(tokens.squeeze(), skip_special_tokens=True)
print(output)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1818: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


This task is based on the WikiSplit dataset consisting of one million English sentences. It is based on edits from Wikipedia’s revision history.


In [13]:
split_sentence("This comedy drama is produced by Tidy , the company she co-founded in 2008 with her husband David Peet , who is managing director .", model_onnx)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1818: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


'This comedy drama is produced by Tidy. She co-founded Tidy in 2008 with her husband David Peet, who is managing director.'

In [14]:
split_sentence("This comedy drama is produced by Tidy , the company she co-founded in 2008 with her husband David Peet , who is managing director .", model)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1818: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


'This comedy drama is produced by Tidy. She co-founded Tidy in 2008 with her husband David Peet, who is managing director.'